# OpenAI Document Search with Langchain

This example shows how to use the Python [langchain](https://python.langchain.com/docs/get_started/introduction) library to run a text-generation request on open-source LLMs and embedding models using the OpenAI SDK, then augment that request using the text stored in a collection of local PDF documents.

**Requirements:**
- As you will accessing the LLMs and embedding models through Vector AI Engineering's Kaleidoscope Service (Vector Inference + Autoscaling), you will need to request a KScope API Key. Visit [https://kscope.vectorinstitute.ai/](https://kscope.vectorinstitute.ai/) and select "Request API Key" to get started.
- After obtaining the `.env` configurations, paste the content into `~/.kscope.env`. This notebook will read these configs from that file.
- (Optional) Upload some pdf files into the `source_documents` subfolder under this notebook. We have already provided some sample pdfs, but feel free to replace these with your own.

## Set up the RAG workflow environment

In [6]:
from getpass import getpass
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.schema import HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


Load config files.

In [7]:
import sys
from pathlib import Path

# Add root folder of the rag_bootcamp repo to PYTHONPATH
current_dir = Path().resolve()
parent_dir = current_dir.parent
sys.path.insert(0, str(parent_dir))


In [9]:
from utils.load_secrets import load_env_file
load_env_file()

Set up some helper functions:

In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

Make sure other necessary items are in place:

In [ ]:
# Look for the source_documents folder and make sure there is at least 1 pdf file here
contains_pdf = False
directory_path = "./source_documents"
if not os.path.exists(directory_path):
    print(f"ERROR: The {directory_path} subfolder must exist under this notebook")
for filename in os.listdir(directory_path):
    contains_pdf = True if ".pdf" in filename else contains_pdf
if not contains_pdf:
    print(f"ERROR: The {directory_path} subfolder must contain at least one .pdf file")

# Uncomment below to use Open-source models hosted on the cluster
## model-names
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
# EMBEDDING_MODEL_NAME = "bge-multilingual-gemma2"
GENERATOR_MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"
EMBEDDING_MODEL_NAME = "bge-multilingual-gemma2"

## OpenAI-comptaible API URL: 
### uncomment the below when using models launched manually via Vector Inference
### Be sure to set these URLs to match the output from "vec-inf status".
# GENERATOR_BASE_URL = "http://gpu900:8080/v1"
# EMBEDDING_BASE_URL = "http://gpu901:8080/v1"

### uncomment the below when using KScope
# GENERATOR_BASE_URL = os.environ.get("OPENAI_BASE_URL")
# EMBEDDING_BASE_URL = os.environ.get("OPENAI_BASE_URL")

## *Alternatively*, Uncomment below when using actual OpenAI models, 
## requires paid API Key and billing setup
# OPENAI_API_KEY = "sk-Your-paid-OpenAI-API-Key"
# GENERATOR_BASE_URL = "https://api.openai.com/v1"
# EMBEDDING_BASE_URL = "https://api.openai.com/v1"
# GENERATOR_MODEL_NAME = "gpt-4o-mini"
# EMBEDDING_MODEL_NAME = "text-embedding-3-small"


## Start with a basic generation request without RAG augmentation

Let's start by asking OpenAI a difficult, domain-specific question we don't expect it to have an answer to. A simple question like "*What is the capital of France?*" is not a good question here, because that's basic knowledge that we expect the LLM to know.

Instead, we want to ask it a question that is very domain-specific that it won't know the answer to. A good example would an obscure detail buried deep within a company's annual report. For example:

"*How many Vector scholarships in AI were awarded in 2022?*"

In [4]:
query = "How many Vector scholarships in AI were awarded in 2022?"

## Now send the query to KScope

In [21]:
llm = ChatOpenAI(model=GENERATOR_MODEL_NAME, base_url=GENERATOR_BASE_URL, api_key=OPENAI_API_KEY)
message = [
    HumanMessage(
        content=query
    )
]
result = llm(message)
print(f"Result: \n\n{result.content}")

NotFoundError: Error code: 404 - {'object': 'error', 'message': 'The model `meta-llama/Meta-Llama-3.1-8B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}

Without additional information, Cohere is unable to answer the question correctly. **Vector in fact awarded 109 AI scholarships in 2022.** Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in the `source_documents` folder. Let's see how we can use RAG to augment our question with a document search and get the correct answer.

## Ingestion: Load and store the documents from source_documents

Start by reading in all the PDF files from `source_documents`, break them up into smaller digestible chunks, then encode them as vector embeddings.

In [9]:
# Load the pdfs
directory_path = "./source_documents"
loader = PyPDFDirectoryLoader(directory_path)
docs = loader.load()
print(f"Number of source documents: {len(docs)}")

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
print(f"Number of text chunks: {len(chunks)}")

# Define the embeddings model
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

print(f"Setting up the embeddings model...")
embeddings = OpenAIEmbeddings(base_url=EMBEDDING_BASE_URL, model=EMBEDDING_MODEL_NAME, api_key=OPENAI_API_KEY)

print(f"Done")

Number of source documents: 42
Number of text chunks: 573
Setting up the embeddings model...
Done


# Retrieval: Make the document chunks available via a retriever

The retriever will identify the document chunks that most closely match our original query. (This takes about 1-2 minutes)

In [13]:
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# Retrieve the most relevant context from the vector store based on the query(No Reranking Applied)
docs = retriever.get_relevant_documents(query)

Let's see what results it found. Important to note, these results are in the order the retriever thought were the best matches.

In [19]:
query = "How many Vector scholarships in AI?"
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

Computer Engineering and Master of Engineering 
(M.Eng) in Electrical and Computer Engineering  — 
Applied Artifcial Intelligence Concentration 
Queen’s University 
• Master of Science (Computer Science, AI)
----------------------------------------------------------------------------------------------------
Document 2:

also actively contributes expertise and insights on 
policy issues related to AI adoption to support the 
best interests of Ontarians and Canadians.
----------------------------------------------------------------------------------------------------
Document 3:

becoming a Faculty Member -- all while having the 
option to start a business or work with some of Canada’s 
largest corporations on AI-driven innovation that will 
enable Canada’s economy to thrive in the future.
----------------------------------------------------------------------------------------------------
Document 4:

about six months of work into about 4-6 weeks and 
helped our team to rapi

These results seem to somewhat match our original query, but we still can't seem to find the information we're looking for. Let's try sending our LLM query again including these results, and see what it comes up with.

In [12]:
print(f"Sending the RAG generation with query: {query}")
qa = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=retriever)
print(f"Result:\n\n{qa.run(query=query)}") 

Sending the RAG generation with query: How many Vector scholarships in AI were awarded in 2022?


/tmp/ipykernel_325932/1674719670.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(f"Result:\n\n{qa.run(query=query)}")


APIConnectionError: Connection error.

# Reranking: Improve the ordering of the document chunks

In [ ]:
embeddings = OpenAIEmbeddings(base_url=EMBEDDING_BASE_URL, model=EMBEDDING_MODEL_NAME, api_key=os.environ["OPENAI_API_KEY"])
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

Now let's see what the reranked results look like:

In [ ]:
pretty_print_docs(compressed_docs)

Lastly, let's run our LLM query a final time with the reranked results:

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=compression_retriever)

print(f"Result:\n\n {qa.run(query=query)}")